In [1]:
import pandas as pd
import yaml
import sys
import os
from glob import glob
from pathlib import Path
from tqdm import tqdm
import pickle
import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings('ignore')

sys.path.append('/kaggle/src')
from utils.xgb import fit_xgb
from utils.metric import compute_comptetition_metric
from utils.postprocess import post_process
from utils.set_seed import seed_base
from feature_engineering.ranker import generate_ranker_features

PACKAGE_DIR = Path("/kaggle/src")
CFG = yaml.safe_load(open(PACKAGE_DIR / "config.yaml", "r"))
print(CFG["ranker"]["execution"]["exp_id"])

CFG["output_dir"] = f"/kaggle/output/{CFG['ranker']['execution']['exp_id']}"
seed_base(CFG["env"]["seed"])

2023-10-04 23:24:00.210804: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-04 23:24:00.277830: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-10-04 23:24:00.710835: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/cuda/lib:/usr/local/lib/x86_64-linux-gnu:/usr/local/nvidia/lib:/u

exp_066


In [2]:
sub = pd.read_csv(os.path.join(CFG["output_dir"], "submission.csv"))
sub.head()

,score,key_step,step,event,level,series_id
0,0.072770,4331.5,4275,onset,13.0,038441c925bb
1,0.745010,4691.5,4635,onset,11.0,038441c925bb
2,2.544872,4811.5,4755,onset,9.0,038441c925bb
3,5.886312,4931.5,4875,onset,7.0,038441c925bb
4,7.134280,4979.5,4923,onset,5.0,038441c925bb


In [3]:
import scipy
score2range = scipy.interpolate.interp1d([-100, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 100], [0, 0, 12, 36, 60, 90, 120, 150, 180, 240, 300, 360, 360])
score2range(1.5)

array(24.)

In [4]:
dfs = []
for _, df in tqdm(sub.groupby("series_id")):
    used = np.zeros(df["step"].max() + 500, dtype=bool)
    df = df.sort_values("score", ascending=False)
    for _, row in df.iterrows():
        if used[row["step"]]:
            continue
        range_ = score2range(row["score"]) 
        used[max(int(row["step"] - range_), 0): int(row["step"] + range_)] = True
        dfs.append(row)
df = pd.DataFrame(dfs)
df.shape

  0%|          | 0/269 [00:00<?, ?it/s]

100%|██████████| 269/269 [00:06<00:00, 41.09it/s]


(71929, 6)

In [5]:
# スコア計算
labels = pd.read_csv(f"{CFG['dataset']['competition_dir']}/train_events.csv").dropna()
score, ap_table = compute_comptetition_metric(labels, df)
print(f"score: {score:.4f}")
display(ap_table)

score: 0.7253


event   tolerance
onset   12           0.133950
        36           0.525654
        60           0.675547
        90           0.757640
        120          0.804428
        150          0.826352
        180          0.840805
        240          0.854650
        300          0.863213
        360          0.871126
wakeup  12           0.158701
        36           0.540973
        60           0.690086
        90           0.777250
        120          0.820131
        150          0.844356
        180          0.859535
        240          0.874794
        300          0.889335
        360          0.896772
dtype: float64